# Neuromodulation example

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion.json``` file in ```data/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026/``` from the xml file we run ```data/convert_sbml_to_json.sh```

To get the RxD functionality of the ```DA_syn``` we must specify the "RxD" block in the connectivity block of the network configuration. See ```data/connectivity.json```

```
    "channel_parameters":
		"RxD": {
		    "species_name": "DA",
		    "flux_variable": "open",
		    "region": "internal",
            "weight_scaling": 1e9,
		},

     ...
     }
```

## Network setup

This simulation models bath appliation of dopamine to the network. The concentrations starts low, then at 2 seconds it increases.

In [1]:
import os
from snudda import Snudda

neuron_path = os.path.join("data", "dspn")
network_path = os.path.join("networks", "neuromodulation_bath")
connectivity_path = os.path.join("data", "connectivity.json")

snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names="dspn", number_of_neurons=[2], 
                      connection_config=connectivity_path, random_seed=12345)

# si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "reaction_diffusion.json"
si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D1.json"

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation"] = "test-modulation.json"
si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Reading connectivity from data/connectivity.json
Adding neurons: dspn from dir data/dspn
Writing networks/neuromodulation_bath/network-config.json
Writing networks/neuromodulation_bath/network-config.json
Placing neurons
Network path: networks/neuromodulation_bath
Creating missing directory networks/neuromodulation_bath/log
Created directory networks/neuromodulation_bath/log
Reading SNUDDA_DATA=None from networks/neuromodulation_bath/network-config.json
No n_putative_points and putative_density, setting n_putative_points = 63
(this must be larger than the number of neurons you want to place)
Generating 63 points for networks/neuromodulation_bath/mesh/Cube-cube-mesh-2.917951293943981e-05.obj
Filtering, keeping inside points: 4 / 26
neuron_name = 'dspn_0', num = 2, neuron_path = 'data/dspn/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026'
stop_parallel disabled, to keep pool running.

Execution time: 0.0s
Touch detection
Network path: networks/neuromodulation_bath
Creating missing director

In [2]:
input_config = os.path.join("data", "input_v5_bath.json")
snudda.setup_input(input_config=input_config)

Setting up inputs, assuming input.json exists
Reading SNUDDA_DATA=None from networks/neuromodulation_bath/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_bath/network-synapses.hdf5
Writing input spikes to networks/neuromodulation_bath/input-spikes.hdf5
Reading SNUDDA_DATA=None from networks/neuromodulation_bath/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_bath/network-synapses.hdf5
!!! Warning, combining definition of cortical_background with cortical_background input for neuron dspn_0 0 (meta modified by input_config)
!!! Warning, combining definition of thalamic_background with thalamic_background input for neuron dspn_0 0 (meta modified by input_config)
!!! Warning, combining definition of cortical with cortical input for neuron dspn_0 0 (meta modified by input_config)
!!! Warning, combining definition of cortical_background with cortical_backgroun

In [5]:
sim_time = 4.0
n_workers = 1
sim_config = "data/DA-bath-experiment.json"
mech_dir = "/home/hjorth/BasalGangliaData/data/neurons/mechanisms"

run_str = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config} --mechdir {mech_dir}"
print(run_str)

mpirun -n 1 snudda simulate networks/neuromodulation_bath --time 4.0 --simulation_config data/DA-bath-experiment.json --mechdir /home/hjorth/BasalGangliaData/data/neurons/mechanisms


In [ ]:
os.system(run_str)

### Add PKA and DA recordings
Here we add recordings from a compartment that receives synaptic input from its neighbour. This has been checked by looking at the synapse_connection matrix (snudda_load command).

## Load the data and plot

In [6]:
from snudda.utils import SnuddaLoadSimulation

output_file = os.path.join(network_path, "simulation", "output.hdf5")
nd = SnuddaLoadSimulation(output_file)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 1)[0][1]
data_da = nd.get_data("DA", 1)[0][1]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/neuromodulation_bath/simulation/output.hdf5


In [7]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()

In [8]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    fig.add_trace(go.Scatter(x=time, y=all_species_data[data_type][0][0].T[0], name=data_type))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()
for data_type in all_species_data:
    yy = all_species_data[data_type][0][0].T[0]
    fig.add_trace(go.Scatter(x=time, y=yy/np.max(yy), name=data_type))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()
for data_type in all_species_data:
    yy = all_species_data[data_type][0][0].T[0]
    fig.add_trace(go.Scatter(x=time, y=yy/yy[0], name=data_type))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
for data_type in all_species_data:
    plt.plot(time, all_species_data[data_type][0][0].T[0], label=data_type)
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
all_species_data[data_type][0][0].T[0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da, label="DA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_pka, label="PKAc")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da_external, label="DA from external?")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
#plt.legend()
plt.title("DA from external?")
plt.show()

In [ ]:
from snudda.plotting import SnuddaPlotSpikeRaster2
fig_file_raster = f"spike-raster.png"

time_range_zoom = (0,0.1)
spr = SnuddaPlotSpikeRaster2(network_path=network_path)

spr.plot_spike_raster(fig_file=fig_file_raster, time_range=time_range_zoom)

In [ ]:
%matplotlib inline

sim_file = os.path.join(network_path, "simulation", "output.hdf5")

from snudda.plotting.plot_traces import PlotTraces
pt = PlotTraces(output_file=sim_file)
# Use trace_id to specify which traces
ax = pt.plot_traces(offset=0, time_range=(0,1),fig_size=(10,4))

In [ ]:
ax0 = pt.plot_traces(offset=0, time_range=(0,1),fig_size=(10,4), trace_id=0)
ax1 = pt.plot_traces(offset=0, time_range=(0,1),fig_size=(10,4), trace_id=1)

In [ ]:
help(pt.plot_traces)

## TODO!! Also add ability to have external synapses affect RxD concentrations

In [ ]:
sim.neurons[0].icell.soma[0](0.5).naf_ms.gbar

In [ ]:
sim.neurons[0].icell.soma[0](0.5).pas.g